# **Free Fall Simulation of 20-Peso Coins on Different Planets in Our Solar System**

## Introduction
### The study of free fall is an essential part of understanding gravitational forces and their effect on objects. Objects of the same mass, such as a coin, will fall at different rates depending on the planet's gravitational pull. Gravitational acceleration is a key factor in determining the speed at which an object falls toward the surface of a celestial body. On Earth, we are accustomed to the acceleration due to gravity being approximately 9.8 m/s². However, this value varies greatly across different planets in our solar system, influencing the rate at which objects fall.



In [ ]:
import pandas as pd

# Planetary gravities (m/s^2)
planetary_gravity = {
    "Mercury": 3.7,
    "Venus": 8.87,
    "Earth": 9.8,
    "Mars": 3.71,
    "Jupiter": 24.79,
    "Saturn": 10.44,
    "Uranus": 8.69,
    "Neptune": 11.15,
}

# Simulation constants
v0 = 0  # Initial velocity (m/s)
y0 = 50  # Initial height (m)
dt = 0.5  # Time step (s)

# Create motion data for each planet
data = []

for planet, gravity in planetary_gravity.items():
    t = 0
    y = y0
    v = v0
    max_time = 20  # Maximum simulation time in seconds
    
    # Variables to store the final values
    final_time = 0
    final_velocity = 0
    
    while y > 0 and t <= max_time:
        # Update velocity and position based on free fall formula
        v = v0 + gravity * t  # Velocity formula: v = v0 + g * t
        y = y0 + 0.5 * gravity * t**2  # Height formula: y = y0 + 0.5 * g * t^2
        
        if y < 0:
            y = 0
        
        t += dt  # Update time step
    
    # After loop ends, we have the final time, final velocity, and gravity
    final_time = t
    final_velocity = v

    # Add data for the planet to the list
    data.append([planet, v0, gravity, final_time, final_velocity])

# Create a DataFrame to represent the results as a table
df = pd.DataFrame(data, columns=["Planet", "Initial Velocity (m/s)", "Gravity (m/s²)", "Final Time (s)", "Final Velocity (m/s)"])

# Display the table
print(df)


In [ ]:
import matplotlib.pyplot as plt

def plot_graph(df):
    """Plot height vs. time for all planets."""
    plt.figure(figsize=(10, 6))
    plt.title("Height vs. Time for Different Planets")
    plt.xlabel("Time (s)")
    plt.ylabel("Height (m)")

    for planet in planetary_gravity.keys():
        planet_data = df[df["Planet"] == planet]
        plt.plot(
            planet_data["Time (s)"],
            planet_data["Height (m)"],
            label=planet,
        )

    plt.legend()
    plt.grid()
    plt.show()

plot_graph(df)


In [ ]:
import pygame

# Initialize Pygame
pygame.init()

# Screen settings
WIDTH, HEIGHT = 1200, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Free-Fall Simulation: Coin Drops on All Planets")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GOLD = (255, 215, 0)

# Fonts
font = pygame.font.SysFont(None, 30)

# Coin properties
coin = {"size": 20, "color": GOLD}

# Physics variables for each planet
planets_data = {
    planet: {"t": 0, "y": 50, "v": 0}
    for planet in planetary_gravity.keys()
}

def draw_coin(screen, x, y, size, color):
    pygame.draw.circle(screen, color, (x, int(y)), size)

def main():
    clock = pygame.time.Clock()
    running = True
    column_count = 2
    row_count = (len(planetary_gravity) + 1) // column_count
    cell_width = WIDTH // column_count
    cell_height = HEIGHT // row_count

    while running:
        screen.fill(WHITE)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        for idx, (planet, gravity) in enumerate(planetary_gravity.items()):
            # Calculate grid position
            row = idx // column_count
            col = idx % column_count
            x_start = col * cell_width
            y_start = row * cell_height

            # Physics update
            planet_data = planets_data[planet]
            planet_data["v"] += gravity * dt
            planet_data["y"] += planet_data["v"] * dt * 20  # Scale for pixels

            # Stop when it hits the ground
            if planet_data["y"] > y_start + cell_height - 200:
                planet_data["y"] = y_start + cell_height - 200

            # Draw ground
            pygame.draw.line(screen, BLACK, (x_start, y_start + cell_height - 200),
                             (x_start + cell_width, y_start + cell_height - 200), 2)

            # Draw coin
            draw_coin(screen, x_start + cell_width // 2, planet_data["y"], coin["size"], coin["color"])

            # Label
            label = font.render(f"{planet} (g = {gravity:.2f} m/s²)", True, BLACK)
            screen.blit(label, (x_start + 10, y_start + 10))

        pygame.display.flip()
        clock.tick(30)

    pygame.quit()

main()
